In [1]:
# Import Libraries

import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
pd.set_option('mode.chained_assignment', None)
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pandas_ta as ta    # https://github.com/twopirllc/pandas-ta

In [2]:
# User Inputs


assetCodes = ['CL=F', 'GC=F', '^RUT', '^GSPC', 'EURUSD=X', 'GBPJPY=X']
assetNames = ['crudeOil', 'Gold', 'Russel2000', 'S&P500', 'EUR-USD', 'GBP-JPY' ]

#assetCodes = ['^RUT', '^GSPC']
#assetNames = ['Russel2000', 'S&P500']

#assetCodes = ['^RUT']
#assetNames = ['Russel2000']

# Date settings
start_date = '2006-06-30'
end_date = '2022-11-01'
training_months = 145

# Time Interval
timePeriod="max"
daily="1d"

# Daily Indicators
dailyEMAShort = 12
dailyEMALong = 26
rsiLength = 14
momLength = 14
rocLength = 21
smaLength = 50
bbandsLength = 30
macdFast = 12
macdSlow = 26
macdSignal = 9

# Portfolio Metrics
initial = 10000
riskFree = 0.01
risk=0.1

# Pct_Change - days ahead
days = 1

# Feature Zscore
rollingWindow = 30

# Feature Normalization
deltaDays = days

In [3]:
# Indicator Names

# Number of Assets
count = len(assetCodes)


# Daily Indicators
dailyEMAShortIndicatorName = 'EMA_'+str(dailyEMAShort)
newDailylyEMAShortIndicatorName = 'EMAShort'

dailyEMALongIndicatorName = 'EMA_'+str(dailyEMALong)
newDailyEMALongIndicatorName = 'EMALong'

rsiIndicatorName = 'RSI_'+str(rsiLength)
newRsiIndicatorName = 'RSIline'

momIndicatorName = 'MOM_'+str(momLength)
newMomIndicatorName = 'MOMline'

rocIndicatorName = 'ROC_'+str(rocLength)
newRocIndicatorName = 'ROCline'

smaIndicatorName = 'SMA_'+str(smaLength)
newSmaIndicatorName = 'SMAline'

bollingerLowerIndicatorName = 'BBL_'+str(bbandsLength)+'_2.0'
newBollingerLowerIndicatorName = 'lowerBB'

bollingerMiddleIndicatorName = 'BBM_'+str(bbandsLength)+'_2.0'
newBollingerMiddleIndicatorName = 'middleBB'

bollingerUpperIndicatorName = 'BBU_'+str(bbandsLength)+'_2.0'
newBollingerUpperIndicatorName = 'upperBB'

bollingerStdIndicatorName = 'BBB_'+str(bbandsLength)+'_2.0'
newBollingerStdIndicatorName = '2stdBB'

macdIndicatorName = "MACD_"+str(macdFast)+"_"+str(macdSlow)+"_"+str(macdSignal)
newMACDIndicatorName = "MACDline"

macdHistogramIndicatorName = "MACDh_"+str(macdFast)+"_"+str(macdSlow)+"_"+str(macdSignal)
newMACDHistogramIndicatorName = "MACDHistogram"

macdSignalIndicatorName = "MACDs_"+str(macdFast)+"_"+str(macdSlow)+"_"+str(macdSignal)
newMACDSignalIndicatorName = "MACDSignal"

featuresRaw = [newDailylyEMAShortIndicatorName, newDailyEMALongIndicatorName, newRsiIndicatorName, newMomIndicatorName, newRocIndicatorName, 
               newSmaIndicatorName, newBollingerLowerIndicatorName, newBollingerMiddleIndicatorName, newBollingerUpperIndicatorName, 
               newBollingerStdIndicatorName, newMACDIndicatorName, newMACDHistogramIndicatorName, newMACDSignalIndicatorName]


# Number of RawFeatures
featuresCount = len(featuresRaw)

In [4]:
# Function to calculate daily Indicators

def dailyIndicators(assetCode, assetName, timePeriod, daily, days, start_date, end_date, dailyEMAShort, dailyEMALong, rsiLength, momLength, rocLength, 
                    smaLength, bbandsLength, macdFast, macdSlow, macdSignal, dailyEMAShortIndicatorName, dailyEMALongIndicatorName,
                    rsiIndicatorName, momIndicatorName, rocIndicatorName, smaIndicatorName, bollingerLowerIndicatorName, bollingerMiddleIndicatorName, bollingerUpperIndicatorName, bollingerStdIndicatorName, macdIndicatorName,
                    macdHistogramIndicatorName, macdSignalIndicatorName, newDailylyEMAShortIndicatorName,
                    newDailyEMALongIndicatorName, newRsiIndicatorName, newMomIndicatorName, newRocIndicatorName,newSmaIndicatorName,
                    newBollingerLowerIndicatorName, newBollingerMiddleIndicatorName, newBollingerUpperIndicatorName,
                    newBollingerStdIndicatorName, newMACDIndicatorName, newMACDHistogramIndicatorName,
                    newMACDSignalIndicatorName):
    
    # Get Daily Asset Data

    dfDaily = pd.DataFrame()
    dfDaily = dfDaily.ta.ticker(assetCode, period=timePeriod, interval=daily)
    dfDaily = dfDaily[(dfDaily.index > start_date) & (dfDaily.index < end_date)]
    dfDaily = dfDaily[(dfDaily.Close > 0)]
    
    # Create Ticker Column
    #dfDaily["Ticker_index"] = assetName
    dfDaily["Ticker"] = assetName
    #dfDaily["Date_index"] = dfDaily.index
    dfDaily["Date"] = dfDaily.index

    # Use the pct_change function to generate returns from close prices
    dfDaily["ActualReturns"] = dfDaily["Close"].pct_change().shift(-days)
    
    # Drop all NaN values from the DataFrame
    dfDaily = dfDaily.dropna()

    # Initialize the new Signal column
    dfDaily['Signal'] = 0.0

    # When Actual Returns are greater than or equal to 0, generate signal to buy asset long
    dfDaily.loc[(dfDaily['ActualReturns'] >= 0), 'Signal'] = 1

    # When Actual Returns are less than 0, generate signal to sell asset short
    dfDaily.loc[(dfDaily['ActualReturns'] < 0), 'Signal'] = -1


    # Create your own Custom Strategy
    CustomStrategyDaily = ta.Strategy(
        name="Daily Indicators",
        description="daily Trading Indicators",
        ta=[
            {"kind": "ema", "length": dailyEMAShort},
            {"kind": "ema", "length": dailyEMALong},
            {"kind": "rsi", "length": rsiLength},
            {"kind": "mom", "length": momLength},
            {"kind": "roc", "length": rocLength},
            {"kind": "sma", "length": smaLength},
            {"kind": "bbands", "length": bbandsLength},
            {"kind": "macd", "fast": macdFast, "slow": macdSlow, "signal": macdSignal},
        ]
    )


    # Run "Custom Daily Strategy"
    dfDaily.ta.strategy(CustomStrategyDaily)
    dfDaily=dfDaily.dropna()
    algoDataDaily = dfDaily[['Ticker', 'Date', 'Close', 'ActualReturns','Signal',dailyEMAShortIndicatorName, dailyEMALongIndicatorName,
                         rsiIndicatorName, momIndicatorName, rocIndicatorName, smaIndicatorName, bollingerLowerIndicatorName,
                         bollingerMiddleIndicatorName, bollingerUpperIndicatorName, bollingerStdIndicatorName,
                         macdIndicatorName, macdHistogramIndicatorName, macdSignalIndicatorName]]

    algoDataDaily = algoDataDaily.rename({dailyEMAShortIndicatorName: newDailylyEMAShortIndicatorName,
                            dailyEMALongIndicatorName: newDailyEMALongIndicatorName,
                            rsiIndicatorName: newRsiIndicatorName,
                            momIndicatorName: newMomIndicatorName,
                            rocIndicatorName: newRocIndicatorName,
                            smaIndicatorName: newSmaIndicatorName,
                            bollingerLowerIndicatorName: newBollingerLowerIndicatorName,
                            bollingerMiddleIndicatorName: newBollingerMiddleIndicatorName,
                            bollingerUpperIndicatorName: newBollingerUpperIndicatorName,
                            bollingerStdIndicatorName: newBollingerStdIndicatorName,
                            macdIndicatorName: newMACDIndicatorName,
                            macdHistogramIndicatorName: newMACDHistogramIndicatorName,
                            macdSignalIndicatorName: newMACDSignalIndicatorName}, axis=1)
    
    return algoDataDaily

In [5]:
# Calculate Feature Z_Scores
def Zscore(algoData, featuresRaw, rollingWindow):

    for i in featuresRaw:
        algoData[i+'_zscore'] = (algoData[i] - algoData[i].rolling(window=rollingWindow).mean())/algoData[i].rolling(window=rollingWindow).std()
        algoData = algoData.ffill(axis = 0)
        
    return algoData


# Normalize Features with Price
def Normalize(algoData, featuresRaw, deltaDays):

    for i in featuresRaw:
        
        if((algoData[i]=='EMAShort') | (algoData[i]=='EMALong') | (algoData[i]=='SMAline')):
        
            algoData[i+'_normal'] = (algoData[i].diff(periods=deltaDays))
            
            
        elif((algoData[i]=='lowerBB') | (algoData[i]=='middleBB') | (algoData[i]=='upperBB')):
            
            algoData[i+'_normal'] = (algoData['Close']/algoData[i])
            
        elif ((algoData[i]=='RSIline') | (algoData[i]=='MOMline') | (algoData[i]=='ROCline') ):
            
            algoData[i+'_normal'] = (algoData[i]/algoData['2stdBB'])
            
        elif ((algoData[i]=='MACDline') | (algoData[i]=='MACDSignal') | (algoData[i]=='MACDHistogram') ):
            
            algoData[i+'_normal'] = (algoData[i])
        elif((algoData[i] == '2stdBB')):
            algoData[i+'_normal'] = (algoData[i]/algoData['Close'])
        
        else:
            algoData[i+'_normal'] = 0
        
        
    algoData = algoData.ffill(axis = 0)
    
    return algoData
   
# Save file
def saveFile(algoData, assetName, days):
    fileName = 'algoData_'+assetName+'_'+str(days)+'dayForward.csv'
    algoData.to_csv(fileName)

In [6]:
# Buy and Hold Strategy Returns

def buyHoldReturns(algoData):

    # Make first return and signal 0 so all cumulative returns start at 1
    algoData["ActualReturns"][0] = 0
    algoData["Signal"][0] = 0

    algoData['cumBuyHoldReturns'] = (1+algoData['ActualReturns']).cumprod()
    
    returns = algoData[['Signal', "ActualReturns", 'cumBuyHoldReturns']]
    
    return returns


In [7]:
# MACD Strategy Function

def macdStrategy(algoData, risk):
    
    signal = [0]
    position = False
    price = 0
    
    for ind in range(1, algoData.shape[0]):
    
        if((algoData['MACDline'][ind] > algoData['MACDSignal'][ind]) & (algoData['MACDline'][ind-1] > algoData['MACDSignal'][ind-1])):
        
            signal.append(1)
            position = True
            price = algoData['Close'][ind]
    
        elif ((algoData['MACDline'][ind] < algoData['MACDSignal'][ind]) & (algoData['MACDline'][ind-1] < algoData['MACDSignal'][ind-1])):

            signal.append(-1)
            position = True
            price = algoData['Close'][ind]
            
        elif ((position == True) & (signal[-1] == 1) & (algoData['Close'][ind] < price*(1-risk))):
            position = False
            signal.append(0)
            price = algoData['Close'][ind]
        elif((position == True) & (signal[-1] == -1) & (algoData['Close'][ind] > price*(1+risk))):
            position = False
            signal.append(0)
            price = algoData['Close'][ind]
            
        else:
        
            signal.append(np.nan)
        
    return signal

In [8]:
# MACD Strategy returns

def macdReturns(algoData, risk):

    # Make first return 0 so all cumulative returns start at 1
    algoData["ActualReturns"][0] = 0

    algoData['MACDStrategy'] = macdStrategy(algoData, risk)
    algoData['MACDStrategy'] = algoData['MACDStrategy'].ffill()
    algoData['MACDStrategyReturns'] = algoData['ActualReturns'] * algoData['MACDStrategy']
    algoData['MACDStrategyReturns'] = algoData['MACDStrategyReturns'].fillna(0)
    algoData['cumMACDReturns'] = (1 + algoData['MACDStrategyReturns']).cumprod()
    
    returns = algoData[['MACDStrategy', 'MACDStrategyReturns', 'cumMACDReturns']]
    
    return returns


In [9]:
# RSI Strategy Function

def rsiSystem(algoData, risk):
    
    signal = [0]
    position = False
    price = 0
    

    for ind in range(1, algoData.shape[0]):

        if ((algoData['RSIline'][ind] > 80) & (algoData['RSIline'][ind-1] < 80) & (position == False)):
            signal.append(-1)
            position = True
            price = algoData['Close'][ind]
        elif ((algoData['RSIline'][ind] < 20) & (algoData['RSIline'][ind-1] > 20) & (position == False)):
            signal.append(1)
            position = True
            price = algoData['Close'][ind]
        elif ((algoData['RSIline'][ind] < 50) & (algoData['RSIline'][ind-1] > 50) & (position == True) & (signal[-1]==1)):
            signal.append(0)
            position = False
            price = algoData['Close'][ind]
        elif ((algoData['RSIline'][ind] > 50) & (algoData['RSIline'][ind-1] < 50) & (position == True) & (signal[-1]==-1)):
            signal.append(0)
            position = False
            price = algoData['Close'][ind]
        elif ((position == True) & (signal[-1] == 1) & (algoData['Close'][ind] < price*(1-risk))):
            position = False
            signal.append(0)
            price = algoData['Close'][ind]
        
        elif ((position == True) & (signal[-1] == -1) & (algoData['Close'][ind] > price*(1+risk))):
            position = False
            signal.append(0)
            price = algoData['Close'][ind]   
            
        else:
            signal.append(np.nan)
            
            
    return signal

In [10]:
# RSI Strategy returns

def rsiReturns(algoData, risk):

    # Make first return 0 so all cumulative returns start at 1
    algoData["ActualReturns"][0] = 0

    algoData['RSIStrategy'] = rsiSystem(algoData, risk)
    algoData['RSIStrategy'] = algoData['RSIStrategy'].ffill()
    algoData['RSIStrategyReturns'] = algoData['ActualReturns'] * algoData['RSIStrategy']
    algoData['RSIRayStrategyReturns'] = algoData['RSIStrategyReturns'].fillna(0)
    algoData['cumRSIReturns'] = (1 + algoData['RSIStrategyReturns']).cumprod()
    
    returns = algoData[['RSIStrategy', 'RSIStrategyReturns', 'cumRSIReturns']]
    
    return returns


In [11]:
# Impulse System Function

def impulseSystem(algoData, risk):
    
    signal = [0]
    position = False
    price = 0
    
    for ind in range(1, algoData.shape[0]):
    
        if((algoData['EMAShort'][ind] > algoData['EMAShort'][ind-1]) & (algoData['EMALong'][ind] > algoData['EMALong'][ind-1]) & (algoData['MACDHistogram'][ind] > algoData['MACDHistogram'][ind-1])):
        
            signal.append(1)
            position = True
            price=algoData['Close'][ind]
    
        elif ((algoData['EMAShort'][ind] < algoData['EMAShort'][ind-1]) & (algoData['EMALong'][ind] < algoData['EMALong'][ind-1]) & (algoData['MACDHistogram'][ind] < algoData['MACDHistogram'][ind-1])):

            signal.append(-1)
            position = True
            price=algoData['Close'][ind]
            
            
            
        elif ((position == True) & (signal[-1] == 1) & (algoData['Close'][ind] < price*(1-risk))):
            position = False
            signal.append(0)
            price = algoData['Close'][ind]
        
        elif ((position == True) & (signal[-1] == -1) & (algoData['Close'][ind] > price*(1+risk))):
            position = False
            signal.append(0)
            price = algoData['Close'][ind]   
    
        else:
        
            signal.append(np.nan)
           
    return signal

In [12]:
# Impulse Strategy returns

def impulseReturns(algoData, risk):

    # Make first return 0 so all cumulative returns start at 1
    algoData["ActualReturns"][0] = 0

    algoData['ImpulseStrategy'] = impulseSystem(algoData, risk)
    algoData['ImpulseStrategy'] = algoData['ImpulseStrategy'].ffill()
    algoData['ImpulseStrategyReturns'] = algoData['ActualReturns'] * algoData['ImpulseStrategy']
    algoData['ImpulseStrategyReturns'] = algoData['ImpulseStrategyReturns'].fillna(0)
    algoData['cumImpulseReturns'] = (1 + algoData['ImpulseStrategyReturns']).cumprod()
    
    returns = algoData[['ImpulseStrategy', 'ImpulseStrategyReturns', 'cumImpulseReturns']]
    
    return returns


In [13]:
# Bollinger Bands Function

def bbStrategy(algoData, risk):
    
    signal = [0]
    position = False
    price=0
    
    for ind in range(1, algoData.shape[0]):
        
        if ((algoData['Close'][ind] > algoData['upperBB'][ind]) & (algoData['Close'][ind-1] < algoData['upperBB'][ind-1]) & (position == False)):
            signal.append(-1)
            position = True
            price = algoData['Close'][ind]
        elif ((algoData['Close'][ind] < algoData['lowerBB'][ind]) & (algoData['Close'][ind-1] > algoData['lowerBB'][ind-1]) & (position == False)):
            signal.append(1)
            position = True
            price = algoData['Close'][ind]
        elif ((algoData['Close'][ind] < algoData['middleBB'][ind]) & (algoData['Close'][ind-1] > algoData['middleBB'][ind-1]) & (position == True)):
            signal.append(0)
            position = False
            price = algoData['Close'][ind]
        elif ((algoData['Close'][ind] > algoData['middleBB'][ind]) & (algoData['Close'][ind-1] < algoData['middleBB'][ind-1]) & (position == True)):
            signal.append(0)
            position = False
            price = algoData['Close'][ind]
            
        elif ((position == True) & (signal[-1] == 1) & (algoData['Close'][ind] < price*(1-risk))):
            position = False
            signal.append(0)
            price = algoData['Close'][ind]
        
        elif ((position == True) & (signal[-1] == -1) & (algoData['Close'][ind] > price*(1+risk))):
            position = False
            signal.append(0)
            price = algoData['Close'][ind]  
        else:
            signal.append(np.nan)
    
    return signal
    
    

In [14]:
# Bollinger Bands Strategy returns

def bollingerReturns(algoData, risk):

    # Make first return 0 so all cumulative returns start at 1
    algoData["ActualReturns"][0] = 0

    # Caluclate BB Strategy
    algoData['BBStrategy'] = bbStrategy(algoData, risk)
    algoData['BBStrategy'] = algoData['BBStrategy'].ffill()
    algoData['BBStrategyReturns'] = algoData['ActualReturns'] * algoData['BBStrategy']
    algoData['BBStrategyReturns'] = algoData['BBStrategyReturns'].fillna(0)
    algoData['cumBBReturns'] = (1 + algoData['BBStrategyReturns']).cumprod()

    returns = algoData[['BBStrategy', 'BBStrategyReturns', 'cumBBReturns']]
    
    return returns


In [15]:
# Calculate all Strategy Returns and place in a dataframe 

def allReturnsData(algoData, risk):
    
    buyHoldReturn = buyHoldReturns(algoData)
    macdReturn = macdReturns(algoData, risk)
    rsiReturn = rsiReturns(algoData, risk)
    impulseReturn = impulseReturns(algoData, risk)
    bollingerReturn = bollingerReturns(algoData, risk)
    
    allReturns = pd.concat([buyHoldReturn, macdReturn, rsiReturn, impulseReturn, bollingerReturn], axis=1)
    allReturns = allReturns.dropna()
    
    return allReturns

In [16]:
# Plot Strategy Returns

def cumulativeStrategyReturnsPlot(allStrategyReturns, assetName, period):

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=allStrategyReturns.index,
            y=allStrategyReturns['cumBuyHoldReturns'],
            name="Buy&Hold",
            line=dict(color="green")
        ))

    fig.add_trace(
        go.Scatter(
            x=allStrategyReturns.index,
            y=allStrategyReturns['cumMACDReturns'],
            name='MACD',
            line=dict(color="red")
        ))


    fig.add_trace(
        go.Scatter(
            x=allStrategyReturns.index,
            y=allStrategyReturns['cumRSIReturns'],
            name='RSI',
            line=dict(color="blue")
        ))

    fig.add_trace(
        go.Scatter(
            x=allStrategyReturns.index,
            y=allStrategyReturns['cumImpulseReturns'],
            name='Impulse System',
            line=dict(color="orange")
        ))

    fig.add_trace(
        go.Scatter(
            x=allStrategyReturns.index,
            y=allStrategyReturns['cumBBReturns'],
            name='Bollinger Bands',
            line=dict(color="purple")
        ))



    fig.update_layout(
        title={
            'text': "Cumulative Strategy Returns",
        },
        template='seaborn',
        xaxis=dict(autorange=True,
                  title_text='Date'),
        yaxis=dict(autorange=True,
                  title_text='Cumulative Returns'),
        legend=dict(
                orientation='h',
                yanchor='bottom',
                y=1,
                xanchor='right',
                x=1
    ))

    fileName = assetName+'_cumStrategyReturns_'+period+'.png'
    
    fig.write_image(fileName)
    
    return fig.show()
    
    

In [17]:
# Descriptive Statistics Function

def descriptiveStats(allStrategyReturns, initial, riskFree, assetName, period):

    # Calculate Descriptive Statistics

    start_date = allStrategyReturns.index.min()
    end_date = allStrategyReturns.index.max()

    start = str(start_date.day)+'-'+str(start_date.month)+'-'+str(start_date.year)
    end = str(end_date.day)+'-'+str(end_date.month)+'-'+str(end_date.year)

    days = (end_date - start_date).days
    years = days/365

    init_investment = initial
    rf = riskFree

    buyHold_start = init_investment
    macd_start = init_investment
    rsi_start = init_investment
    impulse_start = init_investment
    bollinger_start = init_investment

    buyHold_end = round(allStrategyReturns['cumBuyHoldReturns'][-1] * init_investment,2)
    macd_end = round(allStrategyReturns['cumMACDReturns'][-1] * init_investment,2)
    rsi_end = round(allStrategyReturns['cumRSIReturns'][-1] * init_investment,2)
    impulse_end = round(allStrategyReturns['cumImpulseReturns'][-1] * init_investment,2)
    bollinger_end = round(allStrategyReturns['cumBBReturns'][-1] * init_investment,2)

    buyHold_max_dailyReturn = round(allStrategyReturns['ActualReturns'].max(),6)
    macd_max_dailyReturn = round(allStrategyReturns['MACDStrategyReturns'].max(),6)
    rsi_max_dailyReturn = round(allStrategyReturns['RSIStrategyReturns'].max(),6)
    impulse_max_dailyReturn = round(allStrategyReturns['ImpulseStrategyReturns'].max(),6)
    bollinger_max_dailyReturn = round(allStrategyReturns['BBStrategyReturns'].max(),6)

    buyHold_min_dailyReturn = round(allStrategyReturns['ActualReturns'].min(),6)
    macd_min_dailyReturn = round(allStrategyReturns['MACDStrategyReturns'].min(),6)
    rsi_min_dailyReturn = round(allStrategyReturns['RSIStrategyReturns'].min(),6)
    impulse_min_dailyReturn = round(allStrategyReturns['ImpulseStrategyReturns'].min(),6)
    bollinger_min_dailyReturn = round(allStrategyReturns['BBStrategyReturns'].min(),6)

    buyHold_max_drawdown = round(((allStrategyReturns['cumBuyHoldReturns'].min() - allStrategyReturns['cumBuyHoldReturns'].max())/allStrategyReturns['cumBuyHoldReturns'].max()),6)
    macd_max_drawdown = round(((allStrategyReturns['cumMACDReturns'].min() - allStrategyReturns['cumMACDReturns'].max())/allStrategyReturns['cumMACDReturns'].max()),6)
    rsi_max_drawdown = round(((allStrategyReturns['cumRSIReturns'].min() - allStrategyReturns['cumRSIReturns'].max())/allStrategyReturns['cumRSIReturns'].max()),6)
    impulse_max_drawdown = round(((allStrategyReturns['cumImpulseReturns'].min() - allStrategyReturns['cumImpulseReturns'].max())/allStrategyReturns['cumImpulseReturns'].max()),6)
    bollinger_max_drawdown = round(((allStrategyReturns['cumBBReturns'].min() - allStrategyReturns['cumBBReturns'].max())/allStrategyReturns['cumBBReturns'].max()),6)

    
    
    buyHoldSignals = allStrategyReturns.Signal[(allStrategyReturns['Signal'] == 1) | (allStrategyReturns['Signal'] == -1)].count()
    macdSignals = allStrategyReturns.MACDStrategy[(allStrategyReturns['MACDStrategy'] == 1) | (allStrategyReturns['MACDStrategy'] == -1)].count()
    rsiSignals = allStrategyReturns.RSIStrategy[(allStrategyReturns['RSIStrategy'] == 1) | (allStrategyReturns['RSIStrategy'] == -1)].count()
    impulseSignals = allStrategyReturns.ImpulseStrategy[(allStrategyReturns['ImpulseStrategy'] == 1) | (allStrategyReturns['ImpulseStrategy'] == -1)].count()
    bollingerSignals = allStrategyReturns.BBStrategy[(allStrategyReturns['BBStrategy'] == 1) | (allStrategyReturns['BBStrategy'] == -1)].count()
    
   
    
    buyHoldPos = allStrategyReturns.ActualReturns[(allStrategyReturns['ActualReturns'] > 0)].count()
    macdPos = allStrategyReturns.MACDStrategyReturns[(allStrategyReturns['MACDStrategyReturns'] > 0)].count()
    rsiPos = allStrategyReturns.RSIStrategyReturns[(allStrategyReturns['RSIStrategyReturns'] > 0)].count()
    impulsePos = allStrategyReturns.ImpulseStrategyReturns[(allStrategyReturns['ImpulseStrategyReturns'] > 0)].count()
    bollingerPos = allStrategyReturns.BBStrategyReturns[(allStrategyReturns['BBStrategyReturns'] > 0)].count()
    
    
    buyHoldPosPerc = round((buyHoldPos/buyHoldSignals),6)
    macdPosPerc = round((macdPos/macdSignals),6)
    rsiPosPerc = round((rsiPos/rsiSignals),6)
    impulsePosPerc = round((impulsePos/impulseSignals),6)
    bollingerPosPerc = round((bollingerPos/bollingerSignals),6)
    
    
    buyHoldPosSum = allStrategyReturns.ActualReturns[(allStrategyReturns['ActualReturns'] > 0)].sum()
    macdPosSum = allStrategyReturns.MACDStrategyReturns[(allStrategyReturns['MACDStrategyReturns'] > 0)].sum()
    rsiPosSum = allStrategyReturns.RSIStrategyReturns[(allStrategyReturns['RSIStrategyReturns'] > 0)].sum()
    impulsePosSum = allStrategyReturns.ImpulseStrategyReturns[(allStrategyReturns['ImpulseStrategyReturns'] > 0)].sum()
    bollingerPosSum = allStrategyReturns.BBStrategyReturns[(allStrategyReturns['BBStrategyReturns'] > 0)].sum()
    
    buyHoldPosAvg = round((buyHoldPosSum/buyHoldPos),6)
    macdPosAvg = round((macdPosSum/macdPos),6)
    rsiPosAvg = round((rsiPosSum/rsiPos),6)
    impulsePosAvg = round((impulsePosSum/impulsePos),6)
    bollingerPosAvg = round((bollingerPosSum/bollingerPos),6)
    
    
    
    buyHoldNeg = allStrategyReturns.ActualReturns[(allStrategyReturns['ActualReturns'] < 0)].count()
    macdNeg = allStrategyReturns.MACDStrategyReturns[(allStrategyReturns['MACDStrategyReturns'] < 0)].count()
    rsiNeg = allStrategyReturns.RSIStrategyReturns[(allStrategyReturns['RSIStrategyReturns'] < 0)].count()
    impulseNeg = allStrategyReturns.ImpulseStrategyReturns[(allStrategyReturns['ImpulseStrategyReturns'] < 0)].count()
    bollingerNeg = allStrategyReturns.BBStrategyReturns[(allStrategyReturns['BBStrategyReturns'] < 0)].count()
    
    
    buyHoldNegPerc = round((buyHoldNeg/buyHoldSignals),6)
    macdNegPerc = round((macdNeg/macdSignals),6)
    rsiNegPerc = round((rsiNeg/rsiSignals),6)
    impulseNegPerc = round((impulseNeg/impulseSignals),6)
    bollingerNegPerc = round((bollingerNeg/bollingerSignals),6)
    
    
    
    buyHoldNegSum = allStrategyReturns.ActualReturns[(allStrategyReturns['ActualReturns'] < 0)].sum()
    macdNegSum = allStrategyReturns.MACDStrategyReturns[(allStrategyReturns['MACDStrategyReturns'] < 0)].sum()
    rsiNegSum = allStrategyReturns.RSIStrategyReturns[(allStrategyReturns['RSIStrategyReturns'] < 0)].sum()
    impulseNegSum = allStrategyReturns.ImpulseStrategyReturns[(allStrategyReturns['ImpulseStrategyReturns'] < 0)].sum()
    bollingerNegSum = allStrategyReturns.BBStrategyReturns[(allStrategyReturns['BBStrategyReturns'] < 0)].sum()
    
    buyHoldNegAvg = round((buyHoldNegSum/buyHoldNeg),6)
    macdNegAvg = round((macdNegSum/macdNeg),6)
    rsiNegAvg = round((rsiNegSum/rsiNeg),6)
    impulseNegAvg = round((impulseNegSum/impulseNeg),6)
    bollingerNegAvg = round((bollingerNegSum/bollingerNeg),6)
    
    
    
    buyHold_annualReturn = allStrategyReturns['ActualReturns'].apply(lambda x: (1+x)).cumprod().iloc[-1]**(1/years) - 1
    macd_annualReturn = allStrategyReturns['MACDStrategyReturns'].apply(lambda x: (1+x)).cumprod().iloc[-1]**(1/years) - 1
    rsi_annualReturn = allStrategyReturns['RSIStrategyReturns'].apply(lambda x: (1+x)).cumprod().iloc[-1]**(1/years) - 1
    impulse_annualReturn = allStrategyReturns['ImpulseStrategyReturns'].apply(lambda x: (1+x)).cumprod().iloc[-1]**(1/years) - 1
    bollinger_annualReturn = allStrategyReturns['BBStrategyReturns'].apply(lambda x: (1+x)).cumprod().iloc[-1]**(1/years) - 1


    buyHold_annualVol = allStrategyReturns['ActualReturns'].apply(lambda x: np.log(1+x)).std()*np.sqrt(252)
    macd_annualVol = allStrategyReturns['MACDStrategyReturns'].apply(lambda x: np.log(1+x)).std()*np.sqrt(252)
    rsi_annualVol = allStrategyReturns['RSIStrategyReturns'].apply(lambda x: np.log(1+x)).std()*np.sqrt(252)
    impulse_annualVol = allStrategyReturns['ImpulseStrategyReturns'].apply(lambda x: np.log(1+x)).std()*np.sqrt(252)
    bollinger_annualVol = allStrategyReturns['BBStrategyReturns'].apply(lambda x: np.log(1+x)).std()*np.sqrt(252)

    buyHold_Sharpe = round((buyHold_annualReturn-rf)/buyHold_annualVol,2)
    macd_Sharpe = round((macd_annualReturn-rf)/macd_annualVol,2)
    rsi_Sharpe = round((rsi_annualReturn-rf)/rsi_annualVol,2)
    impulse_Sharpe = round((impulse_annualReturn-rf)/impulse_annualVol,2)
    bollinger_Sharpe = round((bollinger_annualReturn-rf)/bollinger_annualVol,2)


    buyHold_variance = allStrategyReturns['ActualReturns'].var()

    macd_covariance = allStrategyReturns['MACDStrategyReturns'].cov(allStrategyReturns['ActualReturns'])
    rsi_covariance = allStrategyReturns['RSIStrategyReturns'].cov(allStrategyReturns['ActualReturns'])
    impulse_covariance = allStrategyReturns['ImpulseStrategyReturns'].cov(allStrategyReturns['ActualReturns'])
    bollinger_covariance = allStrategyReturns['BBStrategyReturns'].cov(allStrategyReturns['ActualReturns'])

    buyHold_beta = 1

    macd_beta = round(macd_covariance/buyHold_variance,2)
    rsi_beta = round(rsi_covariance/buyHold_variance,2)
    impulse_beta = round(impulse_covariance/buyHold_variance,2)
    bollinger_beta = round(bollinger_covariance/buyHold_variance,2)


    # Table of Descriptive Statistics

    head = ['<b>Statistic<b>', '<b>Buy&Hold<b>', '<b>MACD<b>', '<b>RSI<b>', '<b>Impulse<b>', '<b>Bollinger<b>']
    labels = ['<b>Start Date<b>', '<b>End Date<b>','<b>Initial Investment<b>', '<b>Ending Investment<b>','--------------------------',
              '<b>Signals<b>', '<b>Winning Trades<b>', '<b>Losing Trades<b>', '<b>% Winning<b>', '<b>% Losing<b>', 
              '<b>Average Profit<b>','<b>Average Loss<b>', '--------------------------', 
              '<b>Max Daily Return<b>','<b>Min Daily Return<b>', '<b>Max Drawdown<b>', '<b>Annual Return<b>',
              '<b>Annual Volatility<b>', '<b>Sharpe Ratio<b>', '<b>Beta<b>']


    buyHold_stats = [start, end,'${:,}'.format(buyHold_start), '${:,}'.format(buyHold_end), '--------------------------',
                     buyHoldSignals, buyHoldPos, buyHoldNeg,'{:.2%}'.format(buyHoldPosPerc),'{:.2%}'.format(buyHoldNegPerc), 
                     '{:.2%}'.format(buyHoldPosAvg), '{:.2%}'.format(buyHoldNegAvg), '--------------------------', 
                     '{:.2%}'.format(buyHold_max_dailyReturn), '{:.2%}'.format(buyHold_min_dailyReturn), 
                     '{:.2%}'.format(buyHold_max_drawdown), '{:.2%}'.format(buyHold_annualReturn), 
                     '{:.2%}'.format(buyHold_annualVol), buyHold_Sharpe, buyHold_beta]


    macd_stats = [start, end,'${:,}'.format(macd_start), '${:,}'.format(macd_end),'--------------------------', 
                  macdSignals, macdPos, macdNeg,'{:.2%}'.format(macdPosPerc),'{:.2%}'.format(macdNegPerc),
                  '{:.2%}'.format(macdPosAvg), '{:.2%}'.format(macdNegAvg), '--------------------------', 
                  '{:.2%}'.format(macd_max_dailyReturn), '{:.2%}'.format(macd_min_dailyReturn), 
                  '{:.2%}'.format(macd_max_drawdown), '{:.2%}'.format(macd_annualReturn), 
                  '{:.2%}'.format(macd_annualVol), macd_Sharpe, macd_beta]

    rsi_stats = [start, end, '${:,}'.format(rsi_start), '${:,}'.format(rsi_end),'--------------------------', 
                      rsiSignals, rsiPos, rsiNeg, '{:.2%}'.format(rsiPosPerc),
                      '{:.2%}'.format(rsiNegPerc), '{:.2%}'.format(rsiPosAvg), '{:.2%}'.format(rsiNegAvg), 
                      '--------------------------', '{:.2%}'.format(rsi_max_dailyReturn), 
                      '{:.2%}'.format(rsi_min_dailyReturn), '{:.2%}'.format(rsi_max_drawdown),
                      '{:.2%}'.format(rsi_annualReturn), '{:.2%}'.format(rsi_annualVol), 
                      rsi_Sharpe, rsi_beta]


    impulse_stats = [start, end, '${:,}'.format(impulse_start), '${:,}'.format(impulse_end), '--------------------------', 
                     impulseSignals, impulsePos, impulseNeg, '{:.2%}'.format(impulsePosPerc),'{:.2%}'.format(impulseNegPerc), 
                     '{:.2%}'.format(impulsePosAvg), '{:.2%}'.format(impulseNegAvg), '--------------------------', 
                     '{:.2%}'.format(impulse_max_dailyReturn), '{:.2%}'.format(impulse_min_dailyReturn), 
                     '{:.2%}'.format(impulse_max_drawdown), '{:.2%}'.format(impulse_annualReturn), 
                     '{:.2%}'.format(impulse_annualVol), impulse_Sharpe, impulse_beta]


    bollinger_stats = [start, end, '${:,}'.format(bollinger_start), '${:,}'.format(bollinger_end), '--------------------------',
                       bollingerSignals, bollingerPos, bollingerNeg,'{:.2%}'.format(bollingerPosPerc),
                       '{:.2%}'.format(bollingerNegPerc), '{:.2%}'.format(bollingerPosAvg), 
                       '{:.2%}'.format(bollingerNegAvg), '--------------------------', '{:.2%}'.format(bollinger_max_dailyReturn), 
                       '{:.2%}'.format(bollinger_min_dailyReturn), '{:.2%}'.format(bollinger_max_drawdown), 
                       '{:.2%}'.format(bollinger_annualReturn), '{:.2%}'.format(bollinger_annualVol), 
                       bollinger_Sharpe, bollinger_beta]



    fig11 = go.Figure(data=[go.Table(
        header=dict(values=head,
                fill_color='paleturquoise',
                align='left'),
        cells=dict(values=[labels, buyHold_stats, macd_stats, rsi_stats, impulse_stats, bollinger_stats],
            fill_color='lavender',
            align='left'))
    ])

    fig11.update_layout(margin=dict(l=0, r=0, b=0,t=0), width=950, height=650)
    
    
    fileName = assetName+'_descriptiveStatistics_'+period+'.png'
    
    fig11.write_image(fileName)

    return fig11.show()

In [18]:
def dropIndex(algoData):
    algoData = algoData.reset_index(drop=True)
    
    return algoData

In [19]:
# Get Technical Analysis and Signal Data for all Assets

lstdf = [] 

for i in range(0,count):

    algoData = dailyIndicators(assetCodes[i], assetNames[i], timePeriod, daily, days, start_date, end_date, dailyEMAShort, dailyEMALong, rsiLength, momLength, rocLength, 
                    smaLength, bbandsLength, macdFast, macdSlow, macdSignal, dailyEMAShortIndicatorName, dailyEMALongIndicatorName,
                    rsiIndicatorName, momIndicatorName, rocIndicatorName, smaIndicatorName, bollingerLowerIndicatorName, bollingerMiddleIndicatorName, bollingerUpperIndicatorName, bollingerStdIndicatorName, macdIndicatorName,
                    macdHistogramIndicatorName, macdSignalIndicatorName, newDailylyEMAShortIndicatorName,
                    newDailyEMALongIndicatorName, newRsiIndicatorName, newMomIndicatorName, newRocIndicatorName,newSmaIndicatorName,
                    newBollingerLowerIndicatorName, newBollingerMiddleIndicatorName, newBollingerUpperIndicatorName,
                    newBollingerStdIndicatorName, newMACDIndicatorName, newMACDHistogramIndicatorName,
                    newMACDSignalIndicatorName)


    
    # Convert features into rolling z-scores
    algoData = Zscore(algoData, featuresRaw, rollingWindow)
    
    # Normalize features by taking features delta divided by closing price
    algoData = Normalize(algoData, featuresRaw, deltaDays)
    
    # Drop NaN's
    algoData = algoData.dropna()
    

    # Save algoData file
    saveFile(algoData, assetNames[i], days)
    

    
    
    # Split the data into training and test datasets

    # Assign a copy of the aldoData DataFrame
    returns = algoData[algoData.columns]

    # Select the start of the training period
    returns_begin = returns.index.min()

    # Select the ending period for the training data with an offset of x months
    returns_end = returns.index.min() + DateOffset(months=training_months)

    # Generate the taining DataFrames
    returns_train = returns.loc[returns_begin:returns_end]

    # Generate the test DataFrames
    returns_test = returns.loc[returns_end+DateOffset(days=1):]
    
    print(assetNames[i])
    print('------------')
    print('\n')
    
    # Calulate all the Strategy returns for the training period
    allStrategyReturnsTrain = allReturnsData(returns_train, risk)
    
    #print(assetNames[i]+'-Training Period')

    # Plot the all the Strategy returns for the training period
    #cumulativeStrategyReturnsPlot(allStrategyReturnsTrain, assetNames[i], 'training')

    # Descriptive Statistics for all Strategy returns for the training period
    #descriptiveStats(allStrategyReturnsTrain, initial, riskFree, assetNames[i], 'training')
    
    
    # Calulate all the Strategy returns for the test period
    #allStrategyReturnsTest = allReturnsData(returns_test, risk)
    
    #print(assetNames[i]+'-Test Period')

    # Plot the all the Strategy returns for the test period
    #cumulativeStrategyReturnsPlot(allStrategyReturnsTest, assetNames[i], 'test')

    # Descriptive Statistics for all Strategy returns for the test period
    #descriptiveStats(allStrategyReturnsTest, initial, riskFree, assetNames[i], 'test')
    
    # Drop Index
    algoData = dropIndex(algoData)
    
    # Append dataframe
    lstdf.append(algoData)
    
    del algoData
    
    

crudeOil
------------


Gold
------------


Russel2000
------------


S&P500
------------


EUR-USD
------------


GBP-JPY
------------




In [20]:
# Combine individual stocks into a single data frame
dfAllAssets=pd.concat(lstdf, axis= 0)

# Drop the N/As
dfAllAssets = dfAllAssets.dropna()

# Drop the time component of the date
#dfAllAssets=dfAllAssets.sort_values(['Date','Ticker']).set_index(['Date','Ticker'])#.sort_index('Date','Ticker')
dfAllAssets=dfAllAssets.sort_values(['Date','Ticker']).set_index(['Date'])

In [22]:
dfAllAssets.to_csv('dfAllAssets.csv')

In [38]:
Gold = dfAllAssets[dfAllAssets['Ticker'] == 'Gold']

In [ ]:
dfAllAssets.loc[returns_begin:returns_end]

In [39]:
Gold

,Ticker,Close,ActualReturns,Signal,EMAShort,EMALong,RSIline,MOMline,ROCline,SMAline,...,MOMline_normal,ROCline_normal,SMAline_normal,lowerBB_normal,middleBB_normal,upperBB_normal,2stdBB_normal,MACDline_normal,MACDHistogram_normal,MACDSignal_normal
Date,,,,,,,,,,,,,,,,,,,,,
2006-10-10 00:00:00-04:00,Gold,571.900024,0.000699,1.0,580.702381,590.137017,39.601224,-8.099976,-3.067792,608.812004,...,-0.016961,0.003585,-0.002602,-0.002429,-0.001507,-0.000585,3.342556e-04,-0.000247,-0.000029,-0.000218
2006-10-11 00:00:00-04:00,Gold,572.299988,0.006989,1.0,579.409705,588.815756,39.867741,-9.900024,-2.487653,607.226005,...,-0.003145,0.001014,-0.002771,-0.002070,-0.001215,-0.000359,3.086181e-04,0.000050,0.000215,-0.000164
2006-10-12 00:00:00-04:00,Gold,576.299988,0.021690,1.0,578.931287,587.888662,42.595863,-13.100037,-2.172804,605.854004,...,-0.005553,0.000546,-0.002381,-0.001060,-0.001058,-0.001055,1.443915e-05,0.000779,0.000754,0.000025
2006-10-13 00:00:00-04:00,Gold,588.799988,0.010020,1.0,580.449549,587.956167,50.199550,-1.100037,1.639910,604.750004,...,0.020380,0.006475,-0.001875,0.000568,0.000768,0.000969,5.894391e-05,0.002464,0.001952,0.000513
2006-10-16 00:00:00-04:00,Gold,594.700012,-0.008408,-1.0,582.641928,588.455711,53.341100,3.500000,3.174876,603.688004,...,0.007735,0.002581,-0.001786,0.000863,0.001465,0.002067,1.874772e-04,0.002847,0.001871,0.000975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-24 00:00:00-04:00,Gold,1648.699951,0.002487,1.0,1654.931673,1668.748800,43.793335,-72.400024,0.206643,1693.866001,...,-0.018499,0.000844,-0.001818,0.000221,0.000098,-0.000024,-1.510084e-05,0.000286,0.000516,-0.000230
2022-10-25 00:00:00-04:00,Gold,1652.800049,0.006776,1.0,1654.603731,1667.567411,45.123594,-58.599976,1.817286,1691.294001,...,0.008349,0.000974,-0.001556,0.002534,0.000850,-0.000835,-2.056795e-04,0.000516,0.000597,-0.000081
2022-10-26 00:00:00-04:00,Gold,1664.000000,-0.001983,-1.0,1656.049311,1667.303158,48.695582,-47.699951,2.292989,1689.110002,...,0.006550,0.000286,-0.001312,0.002834,0.001067,-0.000700,-2.160145e-04,0.001028,0.000886,0.000141


In [41]:
Gold[returns_begin:returns_end]

,Ticker,Close,ActualReturns,Signal,EMAShort,EMALong,RSIline,MOMline,ROCline,SMAline,...,MOMline_normal,ROCline_normal,SMAline_normal,lowerBB_normal,middleBB_normal,upperBB_normal,2stdBB_normal,MACDline_normal,MACDHistogram_normal,MACDSignal_normal
Date,,,,,,,,,,,,,,,,,,,,,
2006-10-10 00:00:00-04:00,Gold,571.900024,0.000699,1.0,580.702381,590.137017,39.601224,-8.099976,-3.067792,608.812004,...,-0.016961,0.003585,-0.002602,-0.002429,-0.001507,-0.000585,0.000334,-0.000247,-0.000029,-0.000218
2006-10-11 00:00:00-04:00,Gold,572.299988,0.006989,1.0,579.409705,588.815756,39.867741,-9.900024,-2.487653,607.226005,...,-0.003145,0.001014,-0.002771,-0.002070,-0.001215,-0.000359,0.000309,0.000050,0.000215,-0.000164
2006-10-12 00:00:00-04:00,Gold,576.299988,0.021690,1.0,578.931287,587.888662,42.595863,-13.100037,-2.172804,605.854004,...,-0.005553,0.000546,-0.002381,-0.001060,-0.001058,-0.001055,0.000014,0.000779,0.000754,0.000025
2006-10-13 00:00:00-04:00,Gold,588.799988,0.010020,1.0,580.449549,587.956167,50.199550,-1.100037,1.639910,604.750004,...,0.020380,0.006475,-0.001875,0.000568,0.000768,0.000969,0.000059,0.002464,0.001952,0.000513
2006-10-16 00:00:00-04:00,Gold,594.700012,-0.008408,-1.0,582.641928,588.455711,53.341100,3.500000,3.174876,603.688004,...,0.007735,0.002581,-0.001786,0.000863,0.001465,0.002067,0.000187,0.002847,0.001871,0.000975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-10-30 00:00:00-04:00,Gold,1222.599976,-0.008425,-1.0,1223.220388,1215.896177,54.686084,33.299927,2.990481,1204.950000,...,-0.003272,0.000181,0.000586,0.003026,0.001383,-0.000260,-0.000277,-0.000531,-0.000603,0.000072
2018-10-31 00:00:00-04:00,Gold,1212.300049,0.019550,1.0,1221.540336,1215.629797,48.095306,-11.199951,0.823355,1205.344001,...,-0.036707,-0.001788,0.000325,0.000799,0.000389,-0.000021,-0.000069,-0.001166,-0.000991,-0.000175
2018-11-01 00:00:00-04:00,Gold,1236.000000,-0.004126,-1.0,1223.764900,1217.138701,60.031708,17.900024,3.146119,1206.138000,...,0.023544,0.001879,0.000642,0.001368,0.001452,0.001537,0.000008,0.000579,0.000601,-0.000022
